In [ ]:
import os
import keras
from tqdm import tqdm
from glob import glob
import tensorflow as tf
from numpy import zeros
from numpy.random import randint

# Data
from tensorflow.image import resize
from keras.preprocessing.image import load_img, img_to_array

# Data Viz
import matplotlib.pyplot as plt

# Model
from keras.layers import add
from keras.layers import Input
from keras.layers import Layer
from keras.layers import Conv2D
from keras.layers import multiply
from keras.layers import concatenate
from keras.layers import Conv2DTranspose
from keras.layers import MaxPool2D
from keras.layers import UpSampling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import load_model
# Model Functions
from keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.callbacks import Callback, ModelCheckpoint

def load_image(path):
    img = resize(img_to_array(load_img(path))/255., (256,256))
    return img

def load_data(paths):
    images = zeros(shape=(len(paths), 256,256,3))
    masks = zeros(shape=(len(paths), 256,256,3))
    for i, path in tqdm(enumerate(paths), desc="Loading"):
        image = load_image(path)
        images[i] = image
    return images

def show_image(image, title=None):
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')

class Encoder(Layer):

    def __init__(self, filters, rate, pooling=True, **kwargs):
        super(Encoder, self).__init__(**kwargs)

        self.filters = filters
        self.rate = rate
        self.pooling = pooling

        self.bn = BatchNormalization()
        self.c1 = Conv2D(filters, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_normal")
        self.drop = Dropout(rate)
        self.c2 = Conv2D(filters, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_normal")
        self.pool = MaxPool2D()

    def call(self, X):
        x = self.bn(X)
        x = self.c1(x)
        x = self.drop(x)
        x = self.c2(x)
        if self.pooling:
            y = self.pool(x)
            return y, x
        else:
            return x

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "filters":self.filters,
            "rate":self.rate,
            "pooling":self.pooling
        }

class Decoder(Layer):

    def __init__(self, filters, rate, **kwargs):
        super(Decoder, self).__init__(**kwargs)

        self.filters = filters
        self.rate = rate

        self.bn = BatchNormalization()
        self.cT = Conv2DTranspose(filters, kernel_size=3, strides=2, padding='same', activation='relu', kernel_initializer="he_normal")
        self.net = Encoder(filters, rate, pooling=False)

    def call(self, X):
        x, skip_x = X
        x = self.bn(x)
        x = self.cT(x)
        x = concatenate([x, skip_x])
        x = self.net(x)
        return x

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "filters":self.filters,
            "rate":self.rate,
        }

In [ ]:
from PIL import Image

def resize_image(input_path, output_path, target_size):
    # Open the source image
    source_image = Image.open(input_path)

    # Resize the source image to the target size
    resized_image = source_image.resize(target_size, Image.ANTIALIAS)

    # Save the resized image to the output path
    resized_image.save(output_path)

# Example usage
source_image_path = "/content/drive/MyDrive/training_dataset/test/demoans/output12.jpg"
target_image_path = "/content/drive/MyDrive/training_dataset/test/demo/input12.jpg"

# Open the target image to get its dimensions
target_image = Image.open(target_image_path)
target_size = target_image.size

# Resize the source image to match the target size
resize_image(source_image_path, "/content/drive/MyDrive/output12.jpg", target_size)

<ipython-input-27-2c4d9ce1245e>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = source_image.resize(target_size, Image.ANTIALIAS)


In [ ]:
image_path = "drive/MyDrive/training_dataset/test/demo"
total_images = len(os.listdir(image_path))
print(f"Total Number of Images : {total_images}")

all_image_paths = sorted(glob(image_path + "/*.jpg"))

test_paths = all_image_paths[:]
X_test = load_data(test_paths)

unet_inputs = Input(shape=(256,256,3), name="UNetInput")

# Encoder Network : Downsampling phase
p1, c1 = Encoder(64, 0.1, name="Encoder1")(unet_inputs)
p2, c2 = Encoder(128, 0.1, name="Encoder2")(p1)
p3, c3 = Encoder(256, 0.2, name="Encoder3")(p2)
p4, c4 = Encoder(512, 0.2, name="Encoder4")(p3)


# Encoding Layer : Latent Representation
e = Encoder(512, 0.3, pooling=False)(p4)

# Attention + Decoder Network : Upsampling phase.
d1 = Decoder(512, 0.2, name="Decoder1")([e, c4])
d2 = Decoder(256, 0.2, name="Decoder2")([d1, c3])
d3 = Decoder(128, 0.1, name="Decoder3")([d2, c2])
d4 = Decoder(64, 0.1, name="Decoder4")([d3, c1])

# Output
unet_out = Conv2D(3, kernel_size=3, padding='same', activation='sigmoid')(d4)

# Model
UNet = Model(
    inputs=unet_inputs,
    outputs=unet_out,
    name="AttentionUNet"
)

# Compiling
UNet.compile(
    loss='binary_crossentropy',
    optimizer='adam'
)

#load weight

UNet.load_weights('drive/MyDrive/training_dataset/segmodel.h5')

Total Number of Images : 12


Loading: 12it [00:00, 17.79it/s]


In [ ]:
UNet.load_weights('drive/MyDrive/training_dataset/segmodel.h5')

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from random import randint

for id in range(12):
    image = X_test[id]
    pred_mask = UNet.predict(tf.expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:, :, 0] > 0.66)

    plt.figure()  # Create a new figure for each post-processed mask
    plt.imshow(post_process, cmap='gray')  # Display post-processed mask in grayscale
    plt.axis('off')

    plt.savefig(f'/content/drive/MyDrive/training_dataset/test/demoans/output{id+1}.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()


1/1 [==============================] - 2s 2s/step
